In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))
import zipfile
import sys
import time

from keras import backend, models, layers, initializers, regularizers, constraints, optimizers
from keras import callbacks as kc
from keras import optimizers as ko

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import log_loss
import time

['sample_submission_stage_1.csv', 'test_stage_1.tsv', 'trees']


Using TensorFlow backend.


# Load the OOFs and predictions

In [2]:
oof_filenames = os.listdir('oof')

In [3]:
[v for v in oof_filenames if 'bert-base-cased' not in v]

['bnli-mh-bert-base-uncased-seq512-8.csv',
 'bnli-mh-bert-large-cased-seq300-18.csv',
 'bnli-mh-bert-large-uncased-seq300-19.csv',
 'HBnli-mh-bert-base-uncased-seq512-8.csv',
 'HBnli-mh-bert-large-cased-seq300-18.csv',
 'HBnli-mh-bert-large-uncased-seq300-19.csv',
 'Hnaive-bert-base-uncased-seq512-8.csv',
 'Hnaive-bert-large-cased-seq300-18.csv',
 'Hnaive-bert-large-uncased-seq300-19.csv',
 'Hnli-mh-bert-base-uncased-seq512-8.csv',
 'Hnli-mh-bert-large-cased-seq300-18.csv',
 'Hnli-mh-bert-large-uncased-seq300-19.csv',
 'naive-bert-base-uncased-seq512-8.csv',
 'naive-bert-large-cased-seq300-18.csv',
 'naive-bert-large-uncased-seq300-19.csv',
 'nli-mh-bert-base-uncased-seq512-8.csv',
 'nli-mh-bert-large-cased-seq300-18.csv',
 'nli-mh-bert-large-uncased-seq300-19.csv']

In [4]:
def get_tag(oof_name):
    if oof_name.startswith('naive'):
        return '-'.join(oof_name.split('-')[1:4])
    elif oof_name.startswith('nli'):
        return '-'.join(oof_name.split('-')[2:5])
    else:
        pritn("[WTF?]", oof_name)

In [5]:
oofs = [pd.read_csv(os.path.join('oof', v)).values for v in oof_filenames]

dev_filename = "contextual_embeddings_gap_development.json"
val_filename = "contextual_embeddings_gap_validation.json"
test_filename = "contextual_embeddings_gap_test.json"

In [6]:
remove_test = []
remove_validation = []
remove_development = [209, 1506, 1988]

In [7]:
def get_labels():
    dev = pd.read_csv('gap-development.tsv', sep = '\t')
    test =  pd.read_csv('gap-test.tsv', sep = '\t')
    validation =  pd.read_csv('gap-validation.tsv', sep = '\t')
    
    def _get_labels(df):
        labels = df[['A-coref', 'B-coref']].values
        ys = []
        for label in labels:
            y = np.zeros(3)
            if label[0]:
                y[0] = 1
            elif label[1]:
                y[1] = 1
            else:
                y[2] = 1
            ys.append(y)
        return np.array(ys)
        
    dev_labels, test_labels, valid_labels = _get_labels(dev), _get_labels(test), _get_labels(validation)  
    dev_labels = np.delete(dev_labels, remove_development, 0)
    test_labels = np.delete(test_labels, remove_test, 0)
    valid_labels = np.delete(valid_labels, remove_validation, 0)
    print("Shape of test, valid, dev samples", test_labels.shape, valid_labels.shape, dev_labels.shape)
    return test_labels, valid_labels, dev_labels

In [8]:
test_labels, valid_labels, dev_labels = get_labels()

Shape of test, valid, dev samples (2000, 3) (454, 3) (1997, 3)


In [12]:
for oof in oofs:
    print(oof.shape)

(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)
(4451, 3)


In [13]:
train_x = np.concatenate(oofs, axis=1)
train_y = np.concatenate([test_labels, valid_labels, dev_labels], axis=0)

print("train_x shape:", train_x.shape, "train_y shape:", train_y.shape)

train_x shape: (4451, 72) train_y shape: (4451, 3)


In [14]:
import keras
from keras.models import Model
from keras.layers import *
from sklearn.model_selection import KFold
from keras.optimizers import *

In [15]:
def build_mlp_model(feature_nums):
    features_inputs = Input(shape=feature_nums, name='mata-features', dtype="float32")
    features = features_inputs
    
    depth = 5
    for i in range(depth):
        new_features = Dense(24, activation='relu')(features)
        new_features = Dropout(0.5)(new_features)
        features = Concatenate()([features, new_features])

    out_ = Dense(3, activation='softmax')(features)
    
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy',])
    model.summary()
    return model

In [16]:
from keras.callbacks import ReduceLROnPlateau

In [19]:
# Training and cross-validation
folds = KFold(n_splits=7, shuffle=True, random_state=3)
scores = []
oof = np.zeros_like(train_y)

for fold_n, (train_index, valid_index) in enumerate(folds.split(train_x)):
    X_tr, X_val = train_x[train_index], train_x[valid_index]
    Y_tr, Y_val = train_y[train_index], train_y[valid_index]
    
    classif_model = build_mlp_model([train_x.shape[-1]])
    classif_model.compile(optimizer=Adam(lr=1e-3), loss="categorical_crossentropy")

    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True),
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=5e-5),
                kc.ModelCheckpoint('stage_1_finals/nn_ensemble_checkpoint_f{}.pt'.format(fold_n), monitor='val_loss', verbose=0, save_best_only=True, mode='min')]
    # train the model
    classif_model.fit(x=X_tr, y=Y_tr, epochs=200, batch_size=32, 
                      callbacks=callbacks, validation_data=(X_val, Y_val), verbose=1)   
    
    pred_valid = classif_model.predict(x=X_val, verbose=0)
    oof[valid_index] = pred_valid

    # oof[valid_index] = pred_valid.reshape(-1,)
    scores.append(log_loss(Y_val, pred_valid))
    
print("-" * 30)
print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
print(scores)
print("-" * 30)    

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 72)           0                                            
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 24)           1752        mata-features[0][0]              
__________________________________________________________________________________________________
dropout_16 (Dropout)            (None, 24)           0           dense_19[0][0]                   
__________________________________________________________________________________________________
concatenate_16 (Concatenate)    (None, 96)           0           mata-features[0][0]              
                                                                 dropout_16[0][0]                 
__________

3815/3815 [==============================] - 1s 161us/step - loss: 0.2597 - val_loss: 0.2724
Epoch 38/200
3815/3815 [==============================] - 1s 159us/step - loss: 0.2585 - val_loss: 0.2720
Epoch 39/200
3815/3815 [==============================] - 1s 159us/step - loss: 0.2594 - val_loss: 0.2723
Epoch 40/200
3815/3815 [==============================] - 1s 162us/step - loss: 0.2579 - val_loss: 0.2720
Epoch 41/200
3815/3815 [==============================] - 1s 161us/step - loss: 0.2570 - val_loss: 0.2719
Epoch 42/200
3815/3815 [==============================] - 1s 160us/step - loss: 0.2573 - val_loss: 0.2717
Epoch 43/200
3815/3815 [==============================] - 1s 159us/step - loss: 0.2556 - val_loss: 0.2715
Epoch 44/200
3815/3815 [==============================] - 1s 159us/step - loss: 0.2571 - val_loss: 0.2718
Epoch 45/200
3815/3815 [==============================] - 1s 159us/step - loss: 0.2565 - val_loss: 0.2718
Epoch 46/200
3815/3815 [==============================] - 1

Train on 3815 samples, validate on 636 samples
Epoch 1/200
3815/3815 [==============================] - 1s 312us/step - loss: 0.4469 - val_loss: 0.2915
Epoch 2/200
3815/3815 [==============================] - 1s 162us/step - loss: 0.3183 - val_loss: 0.2838
Epoch 3/200
3815/3815 [==============================] - 1s 153us/step - loss: 0.3033 - val_loss: 0.2824
Epoch 4/200
3815/3815 [==============================] - 1s 151us/step - loss: 0.2977 - val_loss: 0.2891
Epoch 5/200
3815/3815 [==============================] - 1s 152us/step - loss: 0.2933 - val_loss: 0.2895
Epoch 6/200
3815/3815 [==============================] - 1s 155us/step - loss: 0.2909 - val_loss: 0.2866
Epoch 7/200
3815/3815 [==============================] - 1s 152us/step - loss: 0.2848 - val_loss: 0.2899
Epoch 8/200
3815/3815 [==============================] - 1s 153us/step - loss: 0.2878 - val_loss: 0.2852
Epoch 9/200
3815/3815 [==============================] - 1s 161us/step - loss: 0.2844 - val_loss: 0.2818
Epoch 10

Train on 3815 samples, validate on 636 samples
Epoch 1/200
3815/3815 [==============================] - 1s 327us/step - loss: 0.4187 - val_loss: 0.3259
Epoch 2/200
3815/3815 [==============================] - 1s 151us/step - loss: 0.3174 - val_loss: 0.3237
Epoch 3/200
3815/3815 [==============================] - 1s 161us/step - loss: 0.2952 - val_loss: 0.3222
Epoch 4/200
3815/3815 [==============================] - 1s 157us/step - loss: 0.2940 - val_loss: 0.3103
Epoch 5/200
3815/3815 [==============================] - 1s 151us/step - loss: 0.2950 - val_loss: 0.3153
Epoch 6/200
3815/3815 [==============================] - 1s 156us/step - loss: 0.2904 - val_loss: 0.3099
Epoch 7/200
3815/3815 [==============================] - 1s 154us/step - loss: 0.2839 - val_loss: 0.3098
Epoch 8/200
3815/3815 [==============================] - 1s 155us/step - loss: 0.2823 - val_loss: 0.3133
Epoch 9/200
3815/3815 [==============================] - 1s 153us/step - loss: 0.2823 - val_loss: 0.3072
Epoch 10

Train on 3815 samples, validate on 636 samples
Epoch 1/200
3815/3815 [==============================] - 1s 333us/step - loss: 0.4134 - val_loss: 0.2854
Epoch 2/200
3815/3815 [==============================] - 1s 181us/step - loss: 0.3200 - val_loss: 0.2723
Epoch 3/200
3815/3815 [==============================] - 1s 151us/step - loss: 0.3053 - val_loss: 0.2624
Epoch 4/200
3815/3815 [==============================] - 1s 150us/step - loss: 0.3009 - val_loss: 0.2604
Epoch 5/200
3815/3815 [==============================] - 1s 149us/step - loss: 0.2988 - val_loss: 0.2579
Epoch 6/200
3815/3815 [==============================] - 1s 158us/step - loss: 0.2934 - val_loss: 0.2584
Epoch 7/200
3815/3815 [==============================] - 1s 150us/step - loss: 0.2881 - val_loss: 0.2624
Epoch 8/200
3815/3815 [==============================] - 1s 153us/step - loss: 0.2893 - val_loss: 0.2582
Epoch 9/200
3815/3815 [==============================] - 1s 152us/step - loss: 0.2841 - val_loss: 0.2610
Epoch 10

Train on 3815 samples, validate on 636 samples
Epoch 1/200
3815/3815 [==============================] - 1s 357us/step - loss: 0.4238 - val_loss: 0.2488
Epoch 2/200
3815/3815 [==============================] - 1s 157us/step - loss: 0.3168 - val_loss: 0.2482
Epoch 3/200
3815/3815 [==============================] - 1s 157us/step - loss: 0.3057 - val_loss: 0.2397
Epoch 4/200
3815/3815 [==============================] - 1s 158us/step - loss: 0.3026 - val_loss: 0.2394
Epoch 5/200
3815/3815 [==============================] - 1s 156us/step - loss: 0.3001 - val_loss: 0.2377
Epoch 6/200
3815/3815 [==============================] - 1s 158us/step - loss: 0.2972 - val_loss: 0.2360
Epoch 7/200
3815/3815 [==============================] - 1s 156us/step - loss: 0.2944 - val_loss: 0.2346
Epoch 8/200
3815/3815 [==============================] - 1s 157us/step - loss: 0.2920 - val_loss: 0.2369
Epoch 9/200
3815/3815 [==============================] - 1s 158us/step - loss: 0.2941 - val_loss: 0.2332
Epoch 10

Epoch 78/200
3815/3815 [==============================] - 1s 148us/step - loss: 0.2634 - val_loss: 0.2267
Epoch 79/200
3815/3815 [==============================] - 1s 150us/step - loss: 0.2658 - val_loss: 0.2268
Epoch 80/200
3815/3815 [==============================] - 1s 148us/step - loss: 0.2624 - val_loss: 0.2270
Epoch 81/200
3815/3815 [==============================] - 1s 147us/step - loss: 0.2632 - val_loss: 0.2270
Epoch 82/200
3815/3815 [==============================] - 1s 148us/step - loss: 0.2664 - val_loss: 0.2273
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 72)           0                                            
__________________________________________________________________________________________________
dense_49 (Dense)                (None, 24)           1752        mata-feat

3815/3815 [==============================] - 1s 156us/step - loss: 0.2617 - val_loss: 0.3134
Epoch 33/200
3815/3815 [==============================] - 1s 155us/step - loss: 0.2602 - val_loss: 0.3131
Epoch 34/200
3815/3815 [==============================] - 1s 155us/step - loss: 0.2607 - val_loss: 0.3132
Epoch 35/200
3815/3815 [==============================] - 1s 154us/step - loss: 0.2595 - val_loss: 0.3137
Epoch 36/200
3815/3815 [==============================] - 1s 155us/step - loss: 0.2593 - val_loss: 0.3135
Epoch 37/200
3815/3815 [==============================] - 1s 156us/step - loss: 0.2600 - val_loss: 0.3134
Epoch 38/200
3815/3815 [==============================] - 1s 155us/step - loss: 0.2585 - val_loss: 0.3136
Epoch 39/200
3815/3815 [==============================] - 1s 156us/step - loss: 0.2577 - val_loss: 0.3135
Epoch 40/200
3815/3815 [==============================] - 1s 158us/step - loss: 0.2577 - val_loss: 0.3138
Epoch 41/200
3815/3815 [==============================] - 1

3816/3816 [==============================] - 1s 156us/step - loss: 0.2855 - val_loss: 0.3074
Epoch 6/200
3816/3816 [==============================] - 1s 156us/step - loss: 0.2866 - val_loss: 0.3092
Epoch 7/200
3816/3816 [==============================] - 1s 156us/step - loss: 0.2849 - val_loss: 0.3096
Epoch 8/200
3816/3816 [==============================] - 1s 155us/step - loss: 0.2803 - val_loss: 0.3128
Epoch 9/200
3816/3816 [==============================] - 1s 155us/step - loss: 0.2790 - val_loss: 0.3100
Epoch 10/200
3816/3816 [==============================] - 1s 156us/step - loss: 0.2797 - val_loss: 0.3113
Epoch 11/200
3816/3816 [==============================] - 1s 155us/step - loss: 0.2760 - val_loss: 0.3083
Epoch 12/200
3816/3816 [==============================] - 1s 155us/step - loss: 0.2743 - val_loss: 0.3078
Epoch 13/200
3816/3816 [==============================] - 1s 155us/step - loss: 0.2719 - val_loss: 0.3069
Epoch 14/200
3816/3816 [==============================] - 1s 15